In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:
test = np.array(test, dtype=np.float32)/255
test = test.reshape(-1,28,28,1)

In [ ]:
x_digit = train.drop(['label'], axis=1)
y_digit = train['label']

In [ ]:
y_digit = keras.utils.to_categorical(y_digit,num_classes=10)
x_digit = np.array(x_digit, dtype=np.float32)/255
x_digit = x_digit.reshape(-1,28,28,1)

In [ ]:
plt.imshow(x_digit[3], cmap='gray')
print(y_digit[3])

## MNIST

In [ ]:
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = mnist.load_data()

In [ ]:
x_mnist = np.concatenate((x_train_mnist,x_test_mnist))
y_mnist = np.concatenate((y_train_mnist,y_test_mnist))

In [ ]:
x_mnist = x_mnist.reshape(-1,28,28,1)
x_mnist = x_mnist.astype(np.float32) / 255
y_mnist = keras.utils.to_categorical(y_mnist,num_classes=10)

## Digit + MNIST

In [ ]:
print(x_digit.shape)
print(y_digit.shape)
print(x_mnist.shape)
print(y_mnist.shape)

In [ ]:
x_train = np.concatenate((x_digit,x_mnist))
y_train = np.concatenate((y_digit,y_mnist))


In [ ]:
print(x_train.shape)
print(y_train.shape)

## creat model

In [ ]:
 def creat_model():
    input_shape = (28,28,1)
    input_layer = Input(input_shape)
    layer = Conv2D(32,(5,5),activation = tf.nn.relu, padding='same', input_shape = input_shape)(input_layer)
    layer = MaxPool2D((2,2))(layer)
    layer = Conv2D(64,(3,3),activation = tf.nn.relu, padding='same')(layer)
    layer = Conv2D(64,(3,3),activation = tf.nn.relu, padding='same')(layer)
    layer = MaxPool2D((2,2))(layer)
    layer = Conv2D(128,(3,3),activation = tf.nn.relu, padding='same')(layer)
    layer = Conv2D(128,(3,3),activation = tf.nn.relu, padding='same')(layer)
    layer = Conv2D(128,(3,3),activation = tf.nn.relu, padding='same')(layer)
    layer = MaxPool2D((2,2))(layer)



    flatten = Flatten()(layer)

    layer = Dense(512,activation = tf.nn.relu)(flatten)
    layer = Dropout(0.25)(layer)
    layer = Dense(512,activation = tf.nn.relu)(layer)
    layer  = Dropout(0.25)(layer)
    layer = Dense(512,activation = tf.nn.relu)(layer)
    layer  = Dropout(0.25)(layer)
    output_layer = Dense(10, activation = tf.nn.softmax)(layer)
    model = Model(input_layer,output_layer)
    
    model.compile(optimizer='Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [ ]:
model = creat_model()
model.summary()

In [ ]:
#Data Augmentation
datagen = ImageDataGenerator(rotation_range=20,
                  width_shift_range=0.20,
                  shear_range=15,
                  zoom_range=0.10,
                  validation_split=0.25,
                  horizontal_flip=False)
datagen.fit(x_train)
generator_train  = datagen.flow(x_train,y_train,batch_size=256,subset='training')
generator_validation  = datagen.flow(x_train,y_train,batch_size=64,subset='validation')

In [ ]:
#ReduceLROnPlateau
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=5 ,min_lr=0.000001,verbose=1)

In [ ]:
#save best wieghts
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath='digit-recognizer-model.hdf5',monitor='val_loss',save_best_only=True,save_weights_only=True,verbose=1)

In [ ]:
epoch = 60
model_hist = model.fit(generator_train,validation_data=generator_validation,epochs=epoch,callbacks=[reduce_lr,checkpoint],verbose=1)

In [ ]:
del model

In [ ]:
#load best weights
model = creat_model()
model.load_weights('digit-recognizer-model.hdf5')

In [ ]:
x = np.arange(1, epoch + 1)
train_acc = model_hist.history['accuracy']
train_loss = model_hist.history['loss']
val_acc = model_hist.history['val_accuracy']
val_loss = model_hist.history['val_loss']

In [ ]:
plt.plot(x , train_acc, x, val_acc)
plt.show()
plt.plot(x , train_loss, x, val_loss)
plt.show()

In [ ]:
prediction = model.predict(test)

In [ ]:
predict = np.array(np.round(prediction), dtype = np.int32)
predict = np.argmax(predict , axis=1).reshape(-1, 1)
out = [{'ImageId': i+1, 'Label': predict[i][0]} for i in range(len(predict))]
pd.DataFrame(out).to_csv('submission.csv', index=False)